In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService

token = '18084cb8c91eee736662c2f23065983acf97af6734e14355c78b8e424c4d0f480efc8b254aff7586b13810c48eb71baf384d26bc382aad7017cec64e857fbae3'

service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='yonsei-dedicated/internal/ymsphdcandid',
    token=token
)

# Or save your credentials on disk.
# QiskitRuntimeService.save_account(channel='ibm_quantum', instance='yonsei-dedicated/internal/ymsphdcandid', token='<IBM Quantum API key>')

In [2]:
from QPCA.decomposition.Qpca import QPCA
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import pickle 
from QPCA.preprocessingUtilities.preprocessing import generate_matrix
from QPCA.benchmark.benchmark import *

-----

qPCA 실행 시 사용하는 큐비트 수에 따른 정확도 향상 수준 평가

In [ ]:
import time
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# 초기 설정
seed = 7000
matrix_dimension = 4
# generate_matrix에 전달할 때는 리스트로 사용
eigenvalues_list_for_matrix = [0.65, 0.25, 0.06, 0.04]
# NumPy 연산을 위해 별도로 배열 생성
eigenvalues_array = np.array(eigenvalues_list_for_matrix)

resolutions = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
n_shots = 50000
n_repetitions = 1

# 리스트를 사용하여 매트릭스 생성
input_matrix = generate_matrix(matrix_dimension=matrix_dimension,
                               replicate_paper=False,
                               seed=seed,
                               eigenvalues_list=eigenvalues_list_for_matrix)

resolution_list = []
errors = []

for res in tqdm(resolutions, desc="Evaluating QPCA accuracy for various resolutions"):
    qpca = QPCA().fit(input_matrix, resolution=res)
    
    estimated_eig, _ = qpca.eigenvectors_reconstruction(n_shots=n_shots,
                                                      n_repetitions=n_repetitions,
                                                      plot_peaks=False)
    # estimated_eig가 리스트라면 NumPy 배열로 변환
    if isinstance(estimated_eig, list):
        estimated_eig = np.array(estimated_eig)
    
    # estimated_eig의 유효성 검사
    if not hasattr(estimated_eig, 'size') or estimated_eig.size != eigenvalues_array.size:
        tqdm.write(f"Resolution: {res} - 예상치 못한 eigenvalue 반환: {estimated_eig}")
        error = np.nan
    else:
        error = np.linalg.norm(estimated_eig - eigenvalues_array)
    
    resolution_list.append(res)
    errors.append(error)
    
    tqdm.write(f"Resolution: {res}, 추정 eigenvalues: {estimated_eig}, 오차: {error:.6f}")
    
# 해상도에 따른 오차 변화 시각화
plt.figure(figsize=(8, 5))
plt.plot(resolution_list, errors, marker='o')
plt.title('해상도에 따른 QPCA eigenvalue 추정 오차')
plt.xlabel('해상도 (Resolution)')
plt.ylabel('추정 오차 (L2 norm)')
plt.grid(True)
plt.show()


Matrix:
 [[0.37 0.18 0.12 0.12]
 [0.18 0.17 0.08 0.09]
 [0.12 0.08 0.13 0.15]
 [0.12 0.09 0.15 0.33]]

eigenvalue: 0.03999999999999997 - eigenvector: [ 0.043  0.284 -0.892  0.348]
eigenvalue: 0.059999999999999866 - eigenvector: [-0.5    0.833  0.182 -0.152]
eigenvalue: 0.24999999999999978 - eigenvector: [-0.564 -0.241  0.194  0.765]
eigenvalue: 0.6499999999999997 - eigenvector: [-0.656 -0.409 -0.364 -0.52 ]


Evaluating QPCA accuracy for various resolutions:   0%|                                         | 0/11 [00:00<?, ?it/s]You chose 2 qubits of resolution. Moreover, since with 2 qubits you have an accuracy of 0.25, if you know that some eigenvalues are smaller or closer to each other than 0.25, please increase the resolution qubits to get better estimates.
The extraction of the eigenvalues is taking longer than expected. You may need to restart the execution increasing the number of resolution qubits and/or the number of shots.
You set an absolute tolerance of 2e-05. If some output eigenvalues are not the expected ones, it is recommended to increase the absolute tolerance to cut away the noisy eigenvalues.
Evaluating QPCA accuracy for various resolutions:   9%|███                              | 1/11 [00:31<05:10, 31.05s/it]You chose 3 qubits of resolution. Moreover, since with 3 qubits you have an accuracy of 0.125, if you know that some eigenvalues are smaller or closer to each other th

Resolution: 2 - 예상치 못한 eigenvalue 반환: []
Resolution: 2, 추정 eigenvalues: [], 오차: nan


You set an absolute tolerance of 2e-05. If some output eigenvalues are not the expected ones, it is recommended to increase the absolute tolerance to cut away the noisy eigenvalues.
Evaluating QPCA accuracy for various resolutions:  18%|██████                           | 2/11 [00:33<02:05, 13.94s/it]You chose 4 qubits of resolution. Moreover, since with 4 qubits you have an accuracy of 0.0625, if you know that some eigenvalues are smaller or closer to each other than 0.0625, please increase the resolution qubits to get better estimates.


Resolution: 3 - 예상치 못한 eigenvalue 반환: [0.625 0.25 ]
Resolution: 3, 추정 eigenvalues: [0.625 0.25 ], 오차: nan


...

Resolution: 8, 추정 eigenvalues: [0.6484375  0.25       0.0390625  0.05859375], 오차: 0.028045


You set an absolute tolerance of 2e-05. If some output eigenvalues are not the expected ones, it is recommended to increase the absolute tolerance to cut away the noisy eigenvalues.
Evaluating QPCA accuracy for various resolutions:  73%|███████████████████████▎        | 8/11 [07:48<05:43, 114.64s/it]

Resolution: 9, 추정 eigenvalues: [0.65039062 0.25       0.06054688 0.0390625 ], 오차: 0.001154


You set an absolute tolerance of 2e-05. If some output eigenvalues are not the expected ones, it is recommended to increase the absolute tolerance to cut away the noisy eigenvalues.
Evaluating QPCA accuracy for various resolutions:  82%|██████████████████████████▏     | 9/11 [51:46<30:06, 903.46s/it]

Resolution: 10, 추정 eigenvalues: [0.65039062 0.25       0.04003906 0.05957031], 오차: 0.027957
